In [2]:
!ssh-keygen -t rsa -b 4096
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
!cat /root/.ssh/id_rsa.pub

In [18]:
#@title Setup Repository
!ssh -T git@github.com
!git clone git@github.com:P2Oileen/oh-my-face.git

!pip install tensorflow-gpu==1.15.2
!pip install ftfy regex tqdm gdown
!pip install git+https://github.com/openai/CLIP.git
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force


     |████████████████████████████████| 410.9 MB 34 kB/s 
     |████████████████████████████████| 503 kB 37.9 MB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
     |████████████████████████████████| 3.8 MB 33.7 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=ffca4685670d32cb7bc5d743190c7ece6243421c56d10341849a7b793564ffb0
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [19]:
#@title Setup Files Downloader
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

download_with_pydrive = True #@param {type:"boolean"}

class Downloader(object):
    def __init__(self, use_pydrive):
        self.use_pydrive = use_pydrive
        current_directory = os.getcwd()
        self.save_dir = os.path.join(os.path.dirname(current_directory), "content/oh-my-face")
        os.makedirs(self.save_dir, exist_ok=True)
        if self.use_pydrive:
            self.authenticate()

    def authenticate(self):
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)

    def download_file(self, file_id, file_name):
        file_dst = f'{self.save_dir}/{file_name}'
        if os.path.exists(file_dst):
            print(f'{file_name} already exists!')
            return
        if self.use_pydrive:
            downloaded = self.drive.CreateFile({'id':file_id})
            downloaded.FetchMetadata(fetch_all=True)
            downloaded.GetContentFile(file_dst)
        else:
            !gdown --id $file_id -O $file_dst

downloader = Downloader(download_with_pydrive)

# Step 1: Select Data Type
Select which experiment you wish to perform inference on:

'face' refers to human face, while 'cat' refers to cat face.

In [6]:
data_type = 'face' #@param ['face', 'cat']

In [7]:
#@title Download Weights from Google Drive
if data_type == 'face':
  downloader.download_file('1efFoGShtZhcd6SCxOPu3AMbKZus478au', 'ffhq.tar.gz')
  downloader.download_file('1bXhWOnwCTTXTz7T7zJ1iXA717tyj-n3U', 'weights-face.tar.gz')
else:
  None
!tar -zxvf oh-my-face/ffhq.tar.gz 
!mv ffhq oh-my-face/
!tar -zxvf oh-my-face/weights-face.tar.gz
!mv weights oh-my-face/src/

ffhq/
ffhq/W.npy
ffhq/S
ffhq/S_mean_std
ffhq/fs3.npy
ffhq/ffhq.pkl
weights/
weights/shape_predictor_68_face_landmarks.dat
weights/flownet.pkl
weights/e4e_ffhq_encode.pt


bin	 datalab  home	 lib64	opt	    root  srv		     tmp    var
boot	 dev	  lib	 media	proc	    run   sys		     tools
content  etc	  lib32  mnt	python-apt  sbin  tensorflow-1.15.2  usr


# Step 2: Upload an image with human face / cat face

Please upload to `oh-my-face/input.jpg`.

In [20]:
#@title Get latent code via Encoder4Editing
os.chdir('/content/oh-my-face/src')
from cog_predict import get_latent_code
import cv2
import argparse
import torch
from PIL import Image

img = cv2.imread('../input.jpg')
latent,img = get_latent_code(img, data_type)
latent = latent.unsqueeze(0)
cv2.imwrite("../input_aligned.jpg",img)
torch.save(latent,"../tmp_latent.pt")

print("Aligned image:")
Image.fromarray(img)

ImportError: ignored

In [25]:
!nvidia-smi

Thu Dec  9 10:29:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    33W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------